In [75]:
%pip install pyexpat
%pip install scikit-learn
%pip install tqdm
%pip install pandas
%pip install spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyexpat (from versions: none)
ERROR: No matching distribution found for pyexpat

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.0/12.8 MB 217.9 kB/s eta 0:00:59
     --------------------------------------- 0.0/12.8 MB 245.8 kB/s eta 0:00:52
     --------------------------------------- 0.1/12.8 MB 328.6 kB/s eta 0:00:39
      --------------------------------------- 0.3/12.8 MB 1.3 MB/s eta 0:00:10
     ---- ----------------------------------- 1.4/12.8 MB 5.0 MB/s eta 0:00:03
     --------- ------------------------------ 2.9/12.8 MB 8.9 MB/s eta 0:00:02
     ------------- -------------------------- 4.4/12.8 MB 11.8 MB/s eta 0:00:01
     ----------------- ---------------------- 5.7/12.8 MB 14.0 MB/s eta 0:00:01
     ---------------------- ----------------- 7.3/12.8 MB 15.6 MB/s eta 0:00:01
     ---------------------------- ----------- 9.1/12.8 MB 1


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import spacy

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [2]:
import json


def preTraitement():
    data_list = []

    with open('goodreads_reviews_spoiler.json', 'r') as fichier_json:
        lines = fichier_json.readlines()  
        for ligne in tqdm(lines, desc='Chargement des données'):
            try:
                objet_json = json.loads(ligne)
                data_list.append(objet_json)
            except json.JSONDecodeError:
                pass

    df = pd.json_normalize(data_list)

    df = df[df['has_spoiler'] == True]

    data_json = df.to_dict(orient='records')

    with open('onlyspoilers.json', 'w') as fichier_resultat:
        json.dump(data_json, fichier_resultat, indent=4)

In [3]:
def spoilersFilter():
    print(bcolors.OKBLUE + "Chargment des données..." + bcolors.ENDC)
    df = pd.read_json('onlyspoilers.json')
    print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

    print(bcolors.OKBLUE + "Filtrage..." + bcolors.ENDC)
    df = df.drop_duplicates(subset=['user_id', 'book_id'])
    df['full_text'] = df['review_sentences'].apply(lambda sentences: ' '.join([sentence[1] for sentence in sentences]))
    df['label'] = df['rating'].apply(lambda x: 1 if x >= 3 else 0)  # 1 pour favorable, 0 pour défavorable
    df = df[['rating', 'label','full_text']]
    df = df.dropna(subset=['rating', 'label', 'full_text'])
    print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

    print(bcolors.OKBLUE + "Ecriture dans data.json..." + bcolors.ENDC)
    data_json = df.to_dict(orient='records')
    with open('data.json', 'w') as fichier_resultat:
        json.dump(data_json, fichier_resultat, indent=4)
    print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

In [4]:
#preTraitement()
#spoilersFilter()

In [9]:
import string


print(bcolors.OKBLUE + "Lecture..." + bcolors.ENDC)
df = pd.read_json('data.json')
df = df.head(500)
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

# Prétraitement avec spaCy
print(bcolors.OKBLUE + "Prétraitement avec spaCy..." + bcolors.ENDC)
punctuations = string.punctuation
nlp = spacy.load("en_core_web_lg", disable=["parser", "ner"])
print(bcolors.OKCYAN + "nlp traitement..." + bcolors.ENDC)
def preprocess(texts):
    return [' '.join([token.lemma_ for token in doc if not token.is_stop and token.lemma_ not in punctuations]) 
            for doc in tqdm(nlp.pipe(texts, batch_size=50), total=len(texts))]
df['processed_text'] = preprocess(df['full_text'].tolist())
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

# Vectorisation
print(bcolors.OKBLUE + "Vectorisation..." + bcolors.ENDC)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['processed_text'])
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

Lecture...
OK
Prétraitement avec spaCy...
nlp traitement...


100%|██████████| 500/500 [00:17<00:00, 28.19it/s]


OK
Vectorisation...
OK


In [10]:
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KNN
print(bcolors.OKBLUE + "KNN..." + bcolors.ENDC)
knn_model = KNeighborsClassifier()
knn_params = {'n_neighbors': [1, 3, 5, 9]}
knn_clf = GridSearchCV(knn_model, knn_params)
knn_clf.fit(X_train, y_train)
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)


# Évaluation de KNN
print(bcolors.OKBLUE + "Évaluation de KNN..." + bcolors.ENDC)
y_pred_knn = knn_clf.predict(X_test)
print("KNN Classification Report")
print(classification_report(y_test, y_pred_knn, zero_division=1))
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

KNN...
OK
Évaluation de KNN...
KNN Classification Report
              precision    recall  f1-score   support

           0       1.00      0.05      0.10        20
           1       0.81      1.00      0.89        80

    accuracy                           0.81       100
   macro avg       0.90      0.53      0.49       100
weighted avg       0.85      0.81      0.73       100

OK


In [11]:
y = df['rating']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# KNN
print(bcolors.OKBLUE + "KNN..." + bcolors.ENDC)
knn_model = KNeighborsClassifier()
knn_params = {'n_neighbors': [1, 3, 5, 9]}
knn_clf = GridSearchCV(knn_model, knn_params)
knn_clf.fit(X_train, y_train)
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)


# Évaluation de KNN
print(bcolors.OKBLUE + "Évaluation de KNN..." + bcolors.ENDC)
y_pred_knn = knn_clf.predict(X_test)
print("KNN Classification Report")
print(classification_report(y_test, y_pred_knn, zero_division=1))
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

KNN...
OK
Évaluation de KNN...
KNN Classification Report
              precision    recall  f1-score   support

           0       1.00      0.00      0.00         1
           1       1.00      0.00      0.00         6
           2       0.00      0.00      1.00        13
           3       0.26      0.47      0.33        15
           4       0.15      0.16      0.15        31
           5       0.36      0.38      0.37        34

    accuracy                           0.25       100
   macro avg       0.46      0.17      0.31       100
weighted avg       0.28      0.25      0.35       100

OK


In [12]:
from sklearn.ensemble import RandomForestClassifier

# RandomForest
print(bcolors.OKBLUE + "RandomForest..." + bcolors.ENDC)
rf_model = RandomForestClassifier(n_jobs=-1)
rf_params = {'n_estimators': [10, 50, 100], 'max_depth': [10, 20, 30]}
rf_clf = GridSearchCV(rf_model, rf_params)
rf_clf.fit(X_train, y_train)
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)

# Évaluation de RandomForest
print(bcolors.OKBLUE + "Évaluation de RandomForest..." + bcolors.ENDC)
y_pred_rf = rf_clf.predict(X_test)
print("RandomForest Classification Report")
print(classification_report(y_test, y_pred_rf, zero_division=1))
print(bcolors.OKGREEN + "OK" + bcolors.ENDC)


RandomForest...
OK
Évaluation de RandomForest...
RandomForest Classification Report
              precision    recall  f1-score   support

           0       1.00      0.00      0.00         1
           1       1.00      0.00      0.00         6
           2       1.00      0.00      0.00        13
           3       0.00      0.00      1.00        15
           4       0.33      1.00      0.50        31
           5       0.33      0.06      0.10        34

    accuracy                           0.33       100
   macro avg       0.61      0.18      0.27       100
weighted avg       0.42      0.33      0.34       100

OK
